**创建一个计算图并在会话中执行**

In [2]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * x * y + y + 2

In [3]:
session = tf.Session()   
session.run(x.initializer)
session.run(y.initializer)
reult = session.run(f)
print(reult)
session.close()

42


In [5]:
# 等同于上面
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [6]:
init = tf.global_variables_initializer()

In [7]:
# 等同于上面
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


**节点值的生命周期**

In [8]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [9]:
with tf.Session() as sess:
    #     此处w, x会计算两次
    print(y.eval())
    print(z.eval())

10
15


In [11]:
with tf.Session() as sess:
    #     此处w, x只计算一次
    print(sess.run([y, z]))

[10, 15]


**用TensorFlow的高级API来训练MLP**

In [12]:
import tensorflow as tf
import numpy as np

In [13]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32).reshape(-1, 28 * 28) / 255.0
x_test = x_test.astype(np.float32).reshape(-1, 28 * 28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
x_valid, x_train = x_train[:5000], x_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

11493376/11490434 [==============================] - 1s 0us/step


In [14]:
feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnn_clf = tf.estimator.DNNClassifier(hidden_units=[300, 100],
                                     n_classes=10,
                                     feature_columns=feature_cols)

input_fn = tf.estimator.inputs.numpy_input_fn(x={"X": x_train},
                                              y=y_train,
                                              num_epochs=40,
                                              batch_size=50,
                                              shuffle=True)
dnn_clf.train(input_fn=input_fn)

W0828 16:36:32.384298 13564 estimator.py:1811] Using temporary folder as model directory: C:\Users\PC-YAO~1\AppData\Local\Temp\tmp09q_voby
W0828 16:36:32.686463 13564 deprecation.py:323] From c:\anaconda3\envs\ml\lib\site-packages\tensorflow\python\training\training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0828 16:36:32.779247 13564 deprecation.py:323] From c:\anaconda3\envs\ml\lib\site-packages\tensorflow_estimator\python\estimator\inputs\queues\feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0828 16:36:32.781231 13564 deprecation.py:323] From c:\anaconda3

In [15]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={"X": x_test},
                                                   y=y_test,
                                                   shuffle=False)
eval_results = dnn_clf.evaluate(input_fn=test_input_fn)

W0828 16:38:29.994055 13564 deprecation.py:323] From c:\anaconda3\envs\ml\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [16]:
eval_results

{'accuracy': 0.98,
 'average_loss': 0.09544891,
 'loss': 12.08214,
 'global_step': 44000}

In [17]:
y_pred_iter = dnn_clf.predict(input_fn=test_input_fn)
y_pred = list(y_pred_iter)
y_pred[0]

{'logits': array([ -9.631214  ,   2.285436  ,   0.09197201,   1.8761902 ,
         -6.2245555 ,  -7.03144   , -22.948462  ,  22.873566  ,
         -4.1801367 ,  -3.4667044 ], dtype=float32),
 'probabilities': array([7.6445774e-15, 1.1446933e-09, 1.2766732e-10, 7.6024870e-10,
        2.3059311e-13, 1.0290127e-13, 1.2581882e-20, 1.0000000e+00,
        1.7812544e-12, 3.6355127e-12], dtype=float32),
 'class_ids': array([7], dtype=int64),
 'classes': array([b'7'], dtype=object),
 'all_class_ids': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'all_classes': array([b'0', b'1', b'2', b'3', b'4', b'5', b'6', b'7', b'8', b'9'],
       dtype=object)}